# SINTA webscraper

Requirements:
- libraries: requests, re, pandas, os, xml.etree.ElementTree
- External program: Download cermine xml reader and put it in the same folder as the code
- edit path to this code in 'mydir' variable

In [1]:
### Program composed with full functions
# This function will convert the pdf directly before change the page and directly erase the pdf after the pages change
# This function comes with debugger for each steps processing for monitoring progresses and errors

# Import libraries
import requests
import re
import pandas
import urllib
import os
import xml.etree.ElementTree as ET

def SINTA_webscraper_v2(journal_id):
    
    # Part 1 journal download
    print()
    page=1 # state initial page
       
    sinta=[]
    link=[]
    pdf_link=[]
    # get to the webpage of journal based on id
    r=requests.get(f'https://sinta.kemdikbud.go.id/journals/profile/{journal_id}?page={page}')
    # get paper view pattern based on inspecting webpage element
    paper_link=re.findall(r'<a href="(.*?)" target="_blank">', r.text)
    # state file address
    mydir='C:/Users/Hadiyan/Documents/CASA/Modules/Docker/Project/web_scraping/'
   
    # create blank DataFrame and its columns
    # get tag line year, authors, title, abstract, keywords, and SINTA level from xml and inputting to DataFrame
    columns = ['year','title','authors', 'abstract', 'keywords','link','sinta_lvl']
    journal_df = pandas.DataFrame(columns=columns)
    
    # This loop logic is while there is still link in the page, go to the next page until there is not
    while paper_link!=[]:
        
        print(f'starting page {page}')
        
        r=requests.get(f'https://sinta.kemdikbud.go.id/journals/profile/{journal_id}?page={page}')
        paper_link = re.findall(r'<a href="(.*?)" target="_blank">', r.text)
        sinta_level = re.findall(r'<div class="stat-num">(.*?)</div>', r.text)
               
                  
        # change from view to download link
        download_link=paper_link
        for i in range(0,len(paper_link)):
            download_link[i]=paper_link[i].replace('view','download')
                        
        # if the link is unlike the standard version that giving an error if exceuted    
            
        # auto download all paper listed in the download link
        # continue downloading loop when the link may be corrupted
        download_count=0
        for i in range(0,len(download_link)):
            try:
                response = urllib.request.urlopen(download_link[i])
                file = open(f'{journal_id}_{i}.pdf','wb')
                file.write(response.read())
                file.close()
                                
                # create list of sinta and paper link based on downloaded article
                link=link+[download_link[i]]
                download_count=download_count+1
                
            except:
                continue
          
        print(f'downloaded {download_count} paper from this page')   
        
        # directly run cermine using command when cermine is already in the same folder
        os.system(r'java -cp cermine-impl-1.13-jar-with-dependencies.jar pl.edu.icm.cermine.ContentExtractor -path C:\Users\Hadiyan\Documents\CASA\Modules\Docker\Project\web_scraping -outputs jats')
        print('start parcing process')
        readlist = [ f for f in os.listdir(mydir) if f.endswith(".cermxml") ]
        for rl in range(0,len(readlist)):
            try:
                
                # Passing the path of the xml document to enable the parsing process
                tree= ET.parse(mydir+readlist[rl])
                # Getting the parent tag of xml document
                root=tree.getroot()
                # Getting pdf informations based on the parent tag
    
                # abstract
                try:
                    abstract_val=root.findall('.//abstract')[0].find('.//p').text
                except:
                    abstract_val=None
                
                #authors
                authors_val=""
                for i in range(0,len(root.findall('.//contrib'))):
                    authors_val += f"{root.findall('.//contrib')[i].find('.//string-name').text}, "
                
                # keywords
                keywords_val=""
                for i in range(0,len(root.findall('.//kwd'))):
                    keywords_val += f"{root.findall('.//kwd')[i].text}, "
        
                # year
                try:
                    published_val=root.findall('.//year')[0].text  
                except:
                    published_val=None
              
                # title
                try:
                    title_val=root.findall('.//article-title')[0].text
                except:
                    title_val=None
                
                # create the list
                journal_list=[published_val,title_val,authors_val,abstract_val,keywords_val,0,0]
               

                # insert the list into DataFrame row
                print(f"parsed loop {rl+1} ")
                 
                # insert list to df
                journal_df.loc[journal_df.shape[0]+1]=journal_list
                
            except:
                continue
              
                
        # get the right shape of link based on how many pdf file successfully converted to xml file
        cermxml_count=[None]*len([f for f in os.listdir(mydir) if f.endswith(".cermxml")])
        link_count=0
        for i in [f for f in os.listdir(mydir) if f.endswith(".cermxml")]:
            cermxml_count[link_count]=download_link[int(i[i.rfind('_')+1])]
            link_count=link_count+1
       
        print(f'cermxml counted is {len(cermxml_count)}')
        print(f'data frame shape is {journal_df.shape[0]}')  
        
        pdf_link=pdf_link+cermxml_count
        print(f'pdf link is {len(pdf_link)}')
        # insert journal link and sinta level to df
        journal_df['link']= pdf_link
        journal_df['sinta_lvl']=[sinta_level[2]]*len(pdf_link)
        
        # delete any old csv, save a new one
        filelist = [ f for f in os.listdir(mydir) if f.endswith(".csv") ]
        for f in filelist:
            os.remove(os.path.join(mydir, f))
        
        journal_df.to_csv(f'journal_{journal_id}_df_page{page}.csv')
        
        # delete pdf and cermxml file in folder
      
        filelist = [ f for f in os.listdir(mydir) if f.endswith(".pdf") ]
        for f in filelist:
            os.remove(os.path.join(mydir, f))

        filelist = [ f for f in os.listdir(mydir) if f.endswith(".cermxml") ]
        for f in filelist:
            os.remove(os.path.join(mydir, f))  
    
        page=page+1
    
    journal_df.to_csv(f'./results_folder/journal_{journal_id}_df_page{page}.csv')
    


    

In [2]:
### run the function to state start and end of journal's id
# start in 11 end in 20
journal_input = []
start=680
end= 1000

for i in range (start,end):
    journal_input.append(i)

In [ ]:
# Very important
# run it and wait until it end
for i in journal_input:
    try:
        SINTA_webscraper_v2(i)
    except:
        print(f'has error in downloading journal {i}')


starting page 1
downloaded 10 paper from this page
start parcing process
parsed loop 1 
parsed loop 2 
parsed loop 3 
parsed loop 4 
parsed loop 5 
parsed loop 6 
parsed loop 7 
parsed loop 8 
parsed loop 9 
parsed loop 10 
cermxml counted is 10
data frame shape is 10
pdf link is 10
starting page 2
downloaded 10 paper from this page
start parcing process
parsed loop 1 
parsed loop 2 
parsed loop 3 
parsed loop 4 
parsed loop 5 
parsed loop 6 
parsed loop 7 
parsed loop 8 
parsed loop 9 
parsed loop 10 
cermxml counted is 10
data frame shape is 20
pdf link is 20
starting page 3
downloaded 10 paper from this page
start parcing process
parsed loop 1 
parsed loop 2 
parsed loop 3 
parsed loop 4 
parsed loop 5 
parsed loop 6 
parsed loop 7 
parsed loop 8 
parsed loop 9 
parsed loop 10 
cermxml counted is 10
data frame shape is 30
pdf link is 30
starting page 4
downloaded 10 paper from this page
start parcing process
parsed loop 1 
parsed loop 2 
parsed loop 3 
parsed loop 4 
parsed loop 5 